In [1]:
import pyspark

In [2]:
from pyspark.sql import HiveContext

In [3]:
import pandas as pd

In [4]:
import pyspark.sql.functions as f

In [5]:
from datetime import datetime

In [6]:
import pytest

In [271]:
!pip install great_expectations

     |████████████████████████████████| 4.5 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 727 kB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 903 kB/s eta 0:00:01
     |████████████████████████████████| 58 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 7.6 MB/s eta 0:00:01     |█████████████████████████████   | 993 kB 7.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 549 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 6.2 MB/s eta 0:00:01     |█████████████████████████████▉  | 2.1 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 737 kB 6.9 MB/s eta 0:0

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=125b7814eca830811094c03e643d82a341d7151179be7c4e0f2076b8c8905026
  Stored in directory: /root/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc
  Created wheel for black: filename=black-20.8b1-py3-none-any.whl size=124184 sha256=2dd63459d2224dbd8262de9fb10079d10d2994542671c212e9ba551c185e9e95
  Stored in directory: /root/.cache/pip/wheels/dd/c8/94/e774504958e4667180d81202d1d45cdc35d6222ae39229871b
Successfully built termcolor black
ERROR: distributed 1.21.8 requires msgpack, which is not installed.
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.2.0
    Uninstalling pyparsing-2.2.0:
      Successfully uninstalled pyparsing-2.2.0
  Attempting uninstall: Click
    Found existing installation: click 6.7
    Uninstalling click-6.7:
      Successfully uninstalled click-6.7
  Attempting uninstall: requests
    Found existing installation: reques

In [7]:
import great_expectations as ge

In [8]:
appName = "PySpark Hive"
master = "local"

spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .enableHiveSupport() \
    .getOrCreate()

In [9]:
sqlContext = HiveContext(sc)

In [10]:
sqlContext.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [11]:
sqlContext.sql("show tables").show()

+--------+-----------+-----------+
|database|  tableName|isTemporary|
+--------+-----------+-----------+
| default|       game|      false|
| default|game_rating|      false|
| default|game_review|      false|
| default|      teste|      false|
+--------+-----------+-----------+



In [12]:
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation", "true")

In [13]:
spark

### Starting the tables

#### Game

In [14]:
sqlContext.sql("DROP TABLE IF EXISTS game PURGE")

DataFrame[]

In [15]:
sqlContext.sql("CREATE TABLE IF NOT EXISTS game(id STRING , name STRING, type STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n'")

DataFrame[]

In [16]:
sqlContext.sql("insert into default.game values ('1', 'game1', 'poker')")
sqlContext.sql("insert into default.game values ('2', 'game2', 'blackjack')")
sqlContext.sql("insert into default.game values ('3', 'game3', 'poker')")

DataFrame[]

#### Game_review

In [17]:
sqlContext.sql("DROP TABLE IF EXISTS game_review PURGE")

DataFrame[]

In [18]:
sqlContext.sql('''CREATE TABLE IF NOT EXISTS game_review(
                review_id STRING , 
                client_id STRING, 
                game_id STRING,
                rating INT,
                review_timestamp TIMESTAMP) 
                PARTITIONED BY (timestamp_review_date STRING) 
                ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' 
                LINES TERMINATED BY '\n' ''')

DataFrame[]

In [19]:
sqlContext.sql('''set hive.exec.dynamic.partition.mode=nonstrict''')

DataFrame[key: string, value: string]

In [20]:
sqlContext.sql('''insert into default.game_review values ('1', '1','1',5,'2020-12-20 00:00:00','2020-12-20'),
                ('2', '2','1',3,'2020-12-20 00:00:00','2020-12-20'),
                ('3', '3','1',2,'2020-12-19 00:00:00','2020-12-19'),
                ('4', '1','1',1,'2020-12-18 00:00:00','2020-12-18'),
                ('5', '2','1',0,'2020-12-20 00:00:00','2020-12-20'),
                ('6', '3','1',5,'2020-12-20 00:00:00','2020-12-20'),
                ('7', '1','2',2,'2020-12-19 00:00:00','2020-12-19'),
                ('8', '2','2',4,'2020-12-17 00:00:00','2020-12-17'),
                ('9', '3','2',4,'2020-12-17 00:00:00','2020-12-17'),
                ('10', '1','2',3,'2019-12-20 00:00:00','2019-12-20'),
                ('11', '2','2',2,'2020-11-20 00:00:00','2020-11-20'),
                ('12', '3','2',5,'2020-10-20 00:00:00','2020-10-20'),
                ('13', '1','2',3,'2020-09-20 00:00:00','2020-09-20'),
                ('14', '2','3',2,'2020-12-20 00:00:00','2020-12-20'),
                ('15', '3','3',1,'2018-12-20 00:00:00','2018-12-20'),
                ('16', '1','3',1,'2020-12-13 00:00:00','2020-12-13'),
                ('17', '2','3',5,'2020-10-01 00:00:00','2020-10-01'),
                ('18', '3','3',5,'2020-11-10 00:00:00','2020-11-10'),
                ('19', '1','3',4,'2020-12-24 00:00:00','2020-12-24'),
                ('20', '2','3',3,'2020-12-25 00:00:00','2020-12-25'),
                ('21', '3','3',2,'2020-12-20 00:00:00','2020-12-20')
            ''')

DataFrame[]

### Treatments

In [21]:
game_df = sqlContext.sql("select * from default.game")

In [22]:
game_review_df = sqlContext.sql('''select * from default.game_review ''')

In [23]:
game_review_df = game_review_df.withColumn("validate_months", 
                f.months_between(f.to_date(f.lit(datetime.now().strftime('%Y-%m-%d'))),
                f.to_date("review_timestamp","yyyy/MM/dd")))

In [24]:
game_review_df

DataFrame[review_id: string, client_id: string, game_id: string, rating: int, review_timestamp: timestamp, timestamp_review_date: string, validate_months: double]

In [25]:
game_review_df = game_review_df.filter(game_review_df.validate_months <= 3)

In [26]:
game_review_df = game_review_df.filter(game_review_df.validate_months >= 0)

In [27]:
game_review_df.show()

+---------+---------+-------+------+-------------------+---------------------+---------------+
|review_id|client_id|game_id|rating|   review_timestamp|timestamp_review_date|validate_months|
+---------+---------+-------+------+-------------------+---------------------+---------------+
|       11|        2|      2|     2|2020-11-20 00:00:00|           2020-11-20|     1.03225806|
|        4|        1|      1|     1|2020-12-18 00:00:00|           2020-12-18|     0.09677419|
|        3|        3|      1|     2|2020-12-19 00:00:00|           2020-12-19|     0.06451613|
|        7|        1|      2|     2|2020-12-19 00:00:00|           2020-12-19|     0.06451613|
|       18|        3|      3|     5|2020-11-10 00:00:00|           2020-11-10|     1.35483871|
|       16|        1|      3|     1|2020-12-13 00:00:00|           2020-12-13|     0.25806452|
|        8|        2|      2|     4|2020-12-17 00:00:00|           2020-12-17|     0.12903226|
|        9|        3|      2|     4|2020-12-17 00:

In [28]:
has_more_than_five_months = game_review_df.groupby(['game_id']).count().where('count > 5')

In [29]:
game_review_df = game_review_df.join(
                has_more_than_five_months,
                ['game_id'])

In [30]:
game_review_df.show()

+-------+---------+---------+------+-------------------+---------------------+---------------+-----+
|game_id|review_id|client_id|rating|   review_timestamp|timestamp_review_date|validate_months|count|
+-------+---------+---------+------+-------------------+---------------------+---------------+-----+
|      1|        1|        1|     5|2020-12-20 00:00:00|           2020-12-20|     0.03225806|    6|
|      1|        2|        2|     3|2020-12-20 00:00:00|           2020-12-20|     0.03225806|    6|
|      1|        5|        2|     0|2020-12-20 00:00:00|           2020-12-20|     0.03225806|    6|
|      1|        6|        3|     5|2020-12-20 00:00:00|           2020-12-20|     0.03225806|    6|
|      1|        3|        3|     2|2020-12-19 00:00:00|           2020-12-19|     0.06451613|    6|
|      1|        4|        1|     1|2020-12-18 00:00:00|           2020-12-18|     0.09677419|    6|
+-------+---------+---------+------+-------------------+---------------------+-------------

In [31]:
game_review_df_full = game_review_df.join(
                    game_df,
                    game_review_df.game_id == game_df.id)

In [32]:
game_review_df_full.show()

+-------+---------+---------+------+-------------------+---------------------+---------------+-----+---+-----+-----+
|game_id|review_id|client_id|rating|   review_timestamp|timestamp_review_date|validate_months|count| id| name| type|
+-------+---------+---------+------+-------------------+---------------------+---------------+-----+---+-----+-----+
|      1|        3|        3|     2|2020-12-19 00:00:00|           2020-12-19|     0.06451613|    6|  1|game1|poker|
|      1|        1|        1|     5|2020-12-20 00:00:00|           2020-12-20|     0.03225806|    6|  1|game1|poker|
|      1|        2|        2|     3|2020-12-20 00:00:00|           2020-12-20|     0.03225806|    6|  1|game1|poker|
|      1|        5|        2|     0|2020-12-20 00:00:00|           2020-12-20|     0.03225806|    6|  1|game1|poker|
|      1|        6|        3|     5|2020-12-20 00:00:00|           2020-12-20|     0.03225806|    6|  1|game1|poker|
|      1|        4|        1|     1|2020-12-18 00:00:00|        

In [33]:
game_review_df_full.createOrReplaceTempView("temp_validated_game_review")

In [34]:
sqlContext.sql("DROP TABLE IF EXISTS game_rating PURGE")

DataFrame[]

In [35]:
sqlContext.sql('''
                 create table game_rating as
                 SELECT 
                  game_id, 
                  name as game_name, 
                  type as game_type,
                  avg(rating) as rating
                  FROM temp_validated_game_review
                  group by
                  game_id,
                  name,
                  type''');

In [36]:
spark.catalog.dropTempView("temp_validated_game_review")

### Validations / Tests

In [37]:
ge_spark_df = sqlContext.sql('''
                 SELECT 
                  game_id, 
                  game_name, 
                  game_type,
                  rating
                  FROM game_rating
                  ''');

In [38]:
ge_pandas = ge_spark_df.toPandas()

In [39]:
ge_df = ge.from_pandas(
    ge_pandas
)

In [40]:
ge_df.expect_table_row_count_to_equal(1)
ge_df.expect_column_values_to_be_in_set("rating",[2.6666666666666665])

{
  "result": {
    "element_count": 1,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "meta": {},
  "success": true
}

In [41]:
validation_results = ge_df.validate()
expectation_suite = ge_df.get_expectation_suite(discard_failed_expectations=False)


In [42]:
if (validation_results.statistics['success_percent'] != 100):
    raise Exception("Data with problem, look at the report: \n"+str(validation_results))